In [2]:
import pandas as pd

In [4]:
communications = pd.read_csv('communications.csv', sep=';')
course_employee_sms = pd.read_csv('course_employee_sms.csv', sep=';')
employees = pd.read_csv('employees.csv', sep=';')
courses_passing = pd.read_csv('courses_passing.csv', sep=';')
courses_info = pd.read_csv('courses_info.csv', sep=';')

In [5]:
communications = pd.read_csv('communications.csv', sep=';')
course_employee_sms = pd.read_csv('course_employee_sms.csv', sep=';')
employees = pd.read_csv('employees.csv', sep=';')
courses_passing = pd.read_csv('courses_passing.csv', sep=';')
courses_info = pd.read_csv('courses_info.csv', sep=';')

In [6]:
communications['communication_dt'] = pd.to_datetime(communications['communication_dt'])
courses_passing['start_dt'] = pd.to_datetime(courses_passing['start_dt'])
courses_passing['last_activity_dt'] = pd.to_datetime(courses_passing['last_activity_dt'])
courses_passing['end_dt'] = pd.to_datetime(courses_passing['end_dt'])

In [7]:
communications_prep = communications.groupby(['employee_id','communication_dt'])['communication_score'].mean().reset_index()

In [8]:
res = communications_prep.merge(courses_passing[courses_passing.pass_frac==1], how='left', on='employee_id')
res = res[res['course_id'].notnull()]
res['before_course'] = res[['end_dt', 'communication_dt']].apply(lambda x: 1 if x.end_dt < x.communication_dt else 0, axis=1)

In [9]:
test5 = res.groupby(['course_id','before_course','employee_id'])['communication_score'].mean().reset_index()

In [10]:
test6 = test5[test5['before_course']==1].merge(test5[test5['before_course']==0], how='left', on=['employee_id','course_id'])

In [11]:
test6['diff']=test6['communication_score_x']-test6['communication_score_y']

In [12]:
test6['target'] = test6['diff'].apply(lambda x: 1 if x>2 else 0)

In [13]:
test6.shape

(8404, 8)

In [14]:
new_df = test6[['course_id','employee_id','target']]

In [15]:
from sklearn import preprocessing

le_user = preprocessing.LabelEncoder()
le_movie = preprocessing.LabelEncoder()
new_df.employee_id = le_user.fit_transform(new_df.employee_id.values)
new_df.course_id = le_movie.fit_transform(new_df.course_id.values)

C:\Users\seles\AppData\Local\Temp\ipykernel_792\1947827498.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.employee_id = le_user.fit_transform(new_df.employee_id.values)
C:\Users\seles\AppData\Local\Temp\ipykernel_792\1947827498.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.course_id = le_movie.fit_transform(new_df.course_id.values)


In [16]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(new_df, test_size=0.2)

In [18]:
import torch
# количество пользователей и продуктов
n_users = new_df['employee_id'].nunique()
n_items = new_df['course_id'].nunique()

# преобразование набора данных в тензор для использования в PyTorch
train_data = torch.FloatTensor(train_df.values)
test_data = torch.FloatTensor(test_df.values)

train_data.shape, test_data.shape, test_data[0]

(torch.Size([6723, 3]), torch.Size([1681, 3]), tensor([  89., 1290.,    0.]))

In [19]:
train_data = torch.tensor(train_data, dtype=torch.long)
test_data = torch.tensor(test_data, dtype=torch.long)

# модель
class RecommenderNet(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=50):
        super().__init__()
        self.user_embedding = torch.nn.Embedding(n_users, n_factors)
        self.item_embedding = torch.nn.Embedding(n_items, n_factors)
        self.fc1 = torch.nn.Linear(n_factors*2, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 1)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, x):
        user_ids = x[:, 1].unsqueeze(1)
        item_ids = x[:, 0].unsqueeze(1)

        user_embedding = self.user_embedding(user_ids)
        item_embedding = self.item_embedding(item_ids)

        x = torch.cat([user_embedding, item_embedding], dim=-1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = torch.nn.ReLU()(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.nn.ReLU()(x)
        x = self.dropout(x)
        x = self.fc3(x)

        return x

n_users = train_data[:, 1].max() + 1
n_items = train_data[:, 0].max() + 1

model = RecommenderNet(n_users, n_items, n_factors=50)
# прогнозы
print(train_data[0].unsqueeze(0))
prediction = model(train_data[0].unsqueeze(0))
print(prediction.squeeze())

tensor([[ 57, 370,   0]])
tensor(-0.1310, grad_fn=<SqueezeBackward0>)


C:\Users\seles\AppData\Local\Temp\ipykernel_792\1206442382.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = torch.tensor(train_data, dtype=torch.long)
C:\Users\seles\AppData\Local\Temp\ipykernel_792\1206442382.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = torch.tensor(test_data, dtype=torch.long)


In [20]:
# оптимизатор и функция потерь
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

In [21]:
train_data.shape

torch.Size([6723, 3])

In [23]:
import numpy as np
epochs = 10

for epoch in range(epochs):
    train_loss = 0.0
    optimizer.zero_grad()
    outputs = model(train_data)
    loss = criterion(outputs.squeeze(), train_data[:, 2].float())
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

    # train RMSE 
    train_rmse = np.sqrt(train_loss / len(train_data))
    print("Epoch:", epoch+1, "Train RMSE:", train_rmse)

Epoch: 1 Train RMSE: 0.00776971531821282
Epoch: 2 Train RMSE: 0.007230829279816047
Epoch: 3 Train RMSE: 0.007120555787702825
Epoch: 4 Train RMSE: 0.007061650337794295
Epoch: 5 Train RMSE: 0.00724552183005187
Epoch: 6 Train RMSE: 0.007192731507762374
Epoch: 7 Train RMSE: 0.007138415780012419
Epoch: 8 Train RMSE: 0.007068753739618254
Epoch: 9 Train RMSE: 0.006934993829415685
Epoch: 10 Train RMSE: 0.006863840247609224


In [24]:
 # test RMSE 
test_loss = 0.0
print(test_data.shape)
outputs = model(test_data)
print(outputs.shape)

loss = criterion(outputs.squeeze(), test_data[:, 2].float())
test_loss += loss.item()

test_rmse = np.sqrt(test_loss / len(test_data))
print("Test RMSE:", test_rmse)

torch.Size([1681, 3])
torch.Size([1681, 1, 1])
Test RMSE: 0.013537137449460853


In [ ]:
# кол-во курсов, которые следует рекомендовать
top_k = 10

# матрицы характеристик пользователя и элемента
user_embeddings = model.user_embedding.weight.detach().numpy()
item_embeddings = model.item_embedding.weight.detach().numpy()

# функция рекомендации
def recommend(user_id):
    # получение курсов с высокими прогнозируемыми рейтингами из курсов, отличных от тех, которые прошел пользователь.
    user_items = new_df[new_df['employee_id'] == user_id]['course_id'].unique()
    user_embedding = user_embeddings[user_id-1]
    scores = np.dot(item_embeddings, user_embedding)
    scores = np.array([(i+1, score) for i, score in enumerate(scores)])
    scores = scores[~np.isin(scores[:, 0], user_items)]
    scores = scores[scores[:, 1].argsort()[::-1]]
    top_k_items = scores[:top_k, 0].astype(int)
    return top_k_items

# вывод примера
user_id = 1
recommended_items = recommend(user_id)
print(f"User {user_id} список рекомендуемых курсов: {recommended_items}")